In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random

categories = ["FoodTrucks", "NotFoodTruck"]
training_data = []

def create_training_data():
    for category in categories:
        class_num = categories.index(category)  # get the classification  (0 or a 1)
        
        for img in os.listdir(os.path.join("Data", category)): 
            try:
                img_array = cv2.imread(os.path.join("Data", category, img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (150, 150))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except OSError as e:
               print("OS Errror", e, os.path.join(path,img))
            except Exception as e:
               print(e, os.path.join(path,img))

create_training_data()

random.shuffle(training_data)

X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, 150, 150, 1)

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten()) 
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.fit(X, y, batch_size = 5, epochs = 3, validation_split = 0.3)
